In [ ]:
import findspark
findspark.init()# the location where you saved Spark bin 
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql.context import SQLContext
spark = SparkSession.builder.appName("HeartPrediction").config("spark.driver.host", "localhost").getOrCreate()

# load a csv file and converting it to a dataframe
data = spark.read.format('com.databricks.spark.csv').option("delimiter", ";").option("header", True).load()#load the location where the dataset is saved 3ndk
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.feature import VectorAssembler
labelIndexer = StringIndexer(inputCol='HeartDisease', outputCol="label").fit(data)
features = data.columns[1:11]

from pyspark.sql.functions import col
dataset = data.select(col('Age').cast('float'),col('Sex'),col('ChestPainType'),
                    col('RestingBP').cast('float'),col('Cholesterol').cast('float'),
                    col('FastingBS').cast('float'),col('RestingECG'),
                    col('MaxHR').cast('float'),col('ExerciseAngina'),
                    col('Oldpeak').cast('float'),col('ST_Slope')
                    ,col('HeartDisease')
                        )

from pyspark.ml.feature import StringIndexer
dataset = StringIndexer(inputCol='HeartDisease',outputCol='label',handleInvalid='keep').fit(dataset).transform(dataset)
dataset = StringIndexer(inputCol='Sex',outputCol='sex',handleInvalid='keep').fit(dataset).transform(dataset)
dataset = StringIndexer(inputCol='ChestPainType',outputCol='Chest_Pain',handleInvalid='keep').fit(dataset).transform(dataset)
dataset = StringIndexer(inputCol='RestingECG',outputCol='Resting_ECG',handleInvalid='keep').fit(dataset).transform(dataset)
dataset = StringIndexer(inputCol='ExerciseAngina',outputCol='Ex_Angina',handleInvalid='keep').fit(dataset).transform(dataset)
dataset = StringIndexer(inputCol='ST_Slope',outputCol='Slope_St',handleInvalid='keep').fit(dataset).transform(dataset)

dataset = dataset.drop('HeartDisease')
dataset = dataset.drop('ChestPainType')
dataset = dataset.drop('RestingECG')
dataset = dataset.drop('ExerciseAngina')
dataset = dataset.drop('ST_Slope')

required_features = ['Age','sex','RestingBP','Cholesterol','FastingBS','MaxHR','Oldpeak','Chest_Pain','Resting_ECG'
                    ,'Ex_Angina','Slope_St']
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=required_features, outputCol='features')
transformed_data = assembler.transform(dataset)

(training_data, test_data) = transformed_data.randomSplit([0.7,0.3])

from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
decisionTree = DecisionTreeClassifier(labelCol='label',featuresCol='features')

model=decisionTree.fit(training_data)
predictions=model.transform(test_data)

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol='label',predictionCol='prediction',metricName='accuracy')

accuracy = evaluator.evaluate(predictions)
print('Test Accuracy: ', accuracy)